In [2]:
import numpy as np

We'd like to compute the $H_k(x)$ for a fixed $x$ for $k=0,...,N$ in a vectorized way.  We could do this using the recurrence relation $H_{n+1}(x) = 2x H_{n}(x) - 2n H_{n-1}(x)$ with $H_0(x) = 1, H_1(x) = 2x$.

In [3]:
def eval_Hermites(x,deg):
    #computes the Hermite polynomials 0,...,deg at an array of points x.
    #returns a matrix of size (len(x),deg)
    H = np.zeros(deg+2)
    H[0] = 1.0
    H[1] = 2*x
    for n in range(2,deg+2):
        H[n] = 2*x*H[n-1]-2*(n-1)*H[n-2]
    return H

In [4]:
x = np.random.randn(1)
deg = 20
coefs = np.random.randn(deg+2)
h_series = np.polynomial.hermite.Hermite(coefs)
print (h_series(x)- np.dot( coefs, eval_Hermites(x,deg))).max()


0.0


We'd also like to compute the derivatives $H_k'(x)$ using the formula $H'_{n}(x) = 2n H_{n-1}(x)$.

In [5]:
derivative_matrix = np.diag( 2*(np.arange(deg+1)+1),k=-1 )
dh_series = h_series.deriv(1)
print dh_series(x)
print np.dot( coefs , derivative_matrix.dot(eval_Hermites(x,deg)) )

[ -1.85045555e+13]
-1.85045554899e+13


Finally, we'd like to compute $x H'_n(x)$

In [6]:
print dh_series(x)*x
print np.dot(coefs , derivative_matrix.dot(eval_Hermites(x,deg))*x )

[ -1.54773971e+13]
-1.54773970758e+13


Assuming a 1D Lagragian of the form $L_\theta(x,\dot{x}) = \sum_{k_1,k_2} \theta_{k_1,k_2} H_{k_1}(x) H_{k_2}(\dot{x})$
we'd like to compute the functional $\ell^1(x,\dot{x},\ddot{x})$ defined such that
$$\langle \ell^1(x,\dot{x},\ddot{x}) , \theta \rangle = \frac{d}{dt} \left( \frac{ \partial L }{\partial \dot{x}} \right)(x,\dot{x},\ddot{x}) - \frac{ \partial L }{\partial x} (x,\dot{x}).$$

In [39]:
def EL_functional(x,v,a, deg_x = 20 , deg_v = 3):
    #computes the functional \ell^x(x,v,a)
    #only works if x,v,a are a single point.
    #you should be able to vector-rize this using parallel for-loops.
    partial_x = np.diag( 2*(np.arange(deg_x+1)+1),k=-1 )
    partial_v = np.diag( 2*(np.arange(deg_v+1)+1),k=-1 )
    h_of_x = eval_Hermites(x,deg_x)
    h_prime_of_x = partial_x.dot(h_of_x)
    h_of_v = eval_Hermites(v,deg_v)
    h_prime_of_v = partial_v.dot(h_of_v)
    h_double_prime_of_v = partial_v.dot(h_prime_of_v)
    t1 = np.kron( h_prime_of_x , h_prime_of_v*v )
    t2 = np.kron( h_of_x , h_double_prime_of_v*a )
    t3 = - np.kron( h_prime_of_x , h_of_v )
    return t1+t2+t3

def EL_functional_2d(x,vx,ax,y,vy,ay,deg_x=20,deg_v=3):
    h_of_x = eval_Hermites(x,deg_x)
    h_of_y = eval_Hermites(y,deg_x)
    h_of_vx = eval_Hermites(vx,deg_v)
    h_of_vy = eval_Hermites(vy,deg_v)
    e1 = reduce( np.kron, [EL_functional(x,vx,ax) , h_of_y , h_of_vy])
    e2 = reduce( np.kron, [h_of_x , h_of_vx , EL_functional(y,vy,ay)])
    return [e1,e2]

Now compute the functionals for a variety of points using parallelization.

In [48]:
#from joblib import Parallel, delayed
N_points = 308
x_arr = np.random.randn(N_points)
dx_arr = np.random.randn(N_points)
ddx_arr = np.random.randn(N_points)
y_arr = np.random.randn(N_points)
dy_arr = np.random.randn(N_points)
ddy_arr = np.random.randn(N_points)

from time import time
arg_gen = zip(x_arr,dx_arr,ddx_arr,y_arr,dy_arr,ddy_arr) 
t0 = time()
ells = []
for arg in arg_gen:
    ells += EL_functional_2d( *arg )

t1 = time()
t = t1-t0
print t
print ells[0].size
print 22*22*5*5

0.615051984787
12100
12100


Now compute $Q = \sum_{(x,v,a)}\ell(x,v,a) \ell(x,v,a)^T$

In [53]:
new_ells = np.stack( ells )   
new_ells.shape
# We want the array \sum_{k} x_i[k] x_j[k]
from time import time
t0 = time()
Q = np.einsum('ki,kj->ij', new_ells, new_ells )
print time() - t0

95.6986968517


With $deg_x = 20, deg_v = 3$ it took 100 seconds to compute Q for 300 points.  We need to compute Q for $N=252954$ which is $84318$ seconds which is $23$ hours

In [58]:
f = lambda x,y: [2*x,3*y]

In [60]:
x,y = f(2,3)
print x
print y

4
9
